# Get Explanation weights - LIME

In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time

from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import accuracy_score 
from sklearn import metrics

from lime import explanation
from lime import lime_base
from utils.lime_timeseries import LimeTimeSeriesExplanation

### 1. Classify by Mr-SEQL

In [9]:
from sktime.classifiers.shapelet_based import MrSEQLClassifier

ds='GunPoint'
ds_dir = 'data'

train_file =  os.path.join(ds_dir, ds, ds+'_TRAIN')
test_file  = os.path.join(ds_dir, ds, ds+'_TEST')

# load train data
train_data = np.genfromtxt(train_file,delimiter=',')

# train_data = np.genfromtxt(train_file)
train_x = train_data[:,1:]
train_y = train_data[:,0]

# Create mrseql object
ms = MrSEQLClassifier(seql_mode='fs')

# Mr-SEQL Custom Configuration
# ms.mode(seql_mode='fs') # to use SEQL as feature selection
# ms.mode(seql_mode='clf') # to use Ensemble SEQL (default)
# ms.symbol_representations(['sax','sfa']) # only sax by default

# Short summary of the methods used.
ms.summary()

# fit training data
ms.fit(train_x,train_y)

# load test data
test_data = np.genfromtxt(test_file,delimiter=',')
test_x = test_data[:,1:]
test_y = test_data[:,0]

# # prediction
predicted = ms.predict(test_x)

# # Classification accuracy
print("Accuracy with mr-seql: %2.4f" % metrics.accuracy_score(test_y, predicted))

Symbolic methods: sax
No symbolic parameters found. To be generated later.
Classification Method: SEQL as feature selection
Accuracy with mr-seql: 0.9867


In [10]:
test_x.shape

(150, 150)

### 2. Get & Save the explanation weights by LIME

In [11]:

num_features = 10
num_slices = 10
num_sample = len(test_y)
explainer = LimeTimeSeriesExplanation(feature_selection='auto')
features = np.empty(dtype=float, shape=(0,num_features))

# for idx in range(num_sample):
for idx in range(num_sample):
    series = test_x[idx, :]
    exp = explainer.explain_instance(series, ms.predict_proba, num_features=num_features, num_samples=500, num_slices=num_slices, 
                                 replacement_method='total_mean', training_set=train_x)
    temp, ans = [], []
    for i in range(num_features):
        feature, weight = exp.as_list()[i]
        temp.append((feature,weight))
    temp.sort()
    for _, val in temp: ans.append(val)
    features = np.append(features, np.array([ans]), axis = 0)    

print(features.shape)



(150, 10)


In [15]:
fileName = 'output/explanation_weight/weights_LIME_%s.txt' % ds

np.savetxt(fileName, features, delimiter=",")